# Black-Scholes-Merton (BSM) Model – Closed-Form

## Introduction
The **Black-Scholes-Merton (BSM) model** is a widely used mathematical framework to
calculate the **theoretical price of European-style options** (calls and puts).
It provides a **closed-form solution** that relates option price to underlying asset
price, strike price, risk-free rate, volatility, and time to expiry.

In the code, the BSM model is used to compute **model prices** for NIFTY and M&M
options, which are then compared with market prices to identify potential
mispricings or arbitrage opportunities.

---

## BSM Closed-Form Formula
For a **call option**:  

$C = S \cdot N(d_1) - K \cdot e^{-rT} \cdot N(d_2)$

For a **put option**:  

$
P = K \cdot e^{-rT} \cdot N(-d_2) - S \cdot N(-d_1)$

Where:  

$
d_1 = \frac{\ln(S/K) + (r + 0.5\sigma^2)T}{\sigma \sqrt{T}}, \quad
d_2 = d_1 - \sigma \sqrt{T}
$

- $S$= Spot price of underlying  
- $K$ = Strike price  
- $r$ = Risk-free interest rate  
- $T$ = Time to expiry (in years)  
- $\sigma$ = Volatility of underlying  
- $N(\cdot)$ = Cumulative standard normal distribution

---

## Advantages
- Provides **analytic (closed-form) prices** for European options.
- Allows quick **calculation of option Greeks** (delta, gamma, vega, theta).
- Useful for **benchmarking market prices** and identifying mispricing.
- Forms the foundation for **implied volatility estimation**.

---

## Limitations
- Assumes **constant volatility** and **risk-free rate**, which is often unrealistic.
- Only valid for **European-style options** (exercise at expiry only).
- Assumes **no dividends** (or must adjust for dividends separately).
- Ignores **market frictions** like transaction costs, liquidity, and bid-ask spreads.
- May not capture **fat tails or jumps** in asset prices.

---

## Practical Use in This Code
- Computes **theoretical model prices** for each option strike.
- Compares **market price vs model price** to generate BUY/SELL/HOLD signals.
- Serves as a **sanity check** for unusual deviations or potential arbitrage.
- Forms the basis for **calculating implied volatility and Greeks** for strategy analysis.


# NSE Options BSM Scanner:

## Overview
This Python script fetches live option chain data from NSE for **NIFTY** and
calculates theoretical option prices using the **Black-Scholes-Merton (BSM) model**.
It compares these model prices to market prices and identifies potential
mispricings or trading signals.

---

## Features
- Computes BSM **call and put prices** for nearest expiry options.
- Estimates ATM **implied volatility** from market mid-prices.
- Calculates option **Greeks**: delta, gamma, vega, theta.
- Generates **BUY/SELL/HOLD signals** based on deviation from model prices.
- Allows scanning a range of strikes around ATM (configurable with `MAX_STRIKES_EACH_SIDE`).
- Outputs CSV containing full option scan.

---

## Data Sources
1. **NSE Option Chain**: via `nsepython`  
2. **Historical underlying price**: via `yfinance` (`^NSEI` for NIFTY)

---

## Usage
1. Set parameters like `SYMBOL`, `RISK_FREE_RATE`, `DECISION_THRESHOLD`, etc.
2. Run the script:
   - Fetches live option chain data
   - Calculates BSM prices and Greeks
   - Generates BUY/SELL/HOLD signals
   - Saves results to CSV (`SAVE_CSV`)
   - Prints top BUY/SELL opportunities in console

---

## Signal Interpretation
- **BUY**: Market price < Model price → option is undervalued
- **SELL**: Market price > Model price → option is overvalued
- **HOLD**: Market price is within threshold of model price
- **diff_pct**: `(Market Price - Model Price) / Model Price` → relative deviation

---

## Outputs
- **CSV file**: Detailed option data with model prices, Greeks, and signals
- **Console summary**: Top BUY/SELL opportunities for quick reference
- **ATM calibrated volatility**: used to price other strikes

---

## Notes
- Threshold for signal generation is configurable via `DECISION_THRESHOLD`.
- Focuses on **NIFTY** by default, but underlying spot and vol can be derived
  from Yahoo Finance if needed.
- Useful for educational, research, and preliminary option strategy analysis.


In [1]:
!pip install nsepython yfinance pandas numpy scipy --quiet

In [2]:
import math
import datetime as dt
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq
from nsepython import nse_optionchain_scrapper
import yfinance as yf

# --------------------------
RISK_FREE_RATE = 0.06
DECISION_THRESHOLD = 0.03
MAX_STRIKES_EACH_SIDE = 6
TRADING_DAYS = 252
# --------------------------

# -------- Black-Scholes Price --------
def bs_price(S, K, r, sigma, T, option_type="call"):
    if T <= 0:
        return max(S - K, 0) if option_type=="call" else max(K - S, 0)
    if sigma <= 0:
        return max(S - K*math.exp(-r*T), 0) if option_type=="call" else max(K*math.exp(-r*T)-S, 0)
    d1 = (math.log(S/K) + (r+0.5*sigma**2)*T)/(sigma*math.sqrt(T))
    d2 = d1 - sigma*math.sqrt(T)
    if option_type=="call":
        return S*norm.cdf(d1) - K*math.exp(-r*T)*norm.cdf(d2)
    else:
        return K*math.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)

# -------- Implied Volatility --------
def implied_volatility(market_price, S, K, r, T, option_type="call"):
    if T <= 0 or market_price <= 0:
        return float("nan")
    def f(sigma):
        return bs_price(S, K, r, sigma, T, option_type) - market_price
    try:
        return brentq(f, 1e-6, 5.0, maxiter=100)
    except Exception:
        return float("nan")

# -------- Trading Signal --------
def decide_trade(market_price, model_price, threshold):
    if model_price == 0 or math.isnan(model_price):
        return "HOLD", None
    diff = (market_price - model_price)/model_price
    if diff > threshold:
        return "SELL", diff
    elif diff < -threshold:
        return "BUY", diff
    else:
        return "HOLD", diff

# -------- Historical Volatility from Yahoo Finance --------
def historical_vol_from_yfinance(ticker, lookback_days=90):
    try:
        end = dt.date.today()
        start = end - dt.timedelta(days=lookback_days)
        df = yf.download(ticker, start=start.isoformat(), end=end.isoformat(), auto_adjust=False, progress=False)
        adj = df["Adj Close"].dropna()
        logret = np.log(adj/adj.shift(1)).dropna()
        daily_std = logret.std(ddof=1)
        ann_vol = daily_std*math.sqrt(TRADING_DAYS)
        spot = float(adj.iloc[-1])
        return spot, float(ann_vol)
    except Exception:
        return None, None

# -------- BSM Greeks --------
def bs_greeks(S, K, r, sigma, T, option_type="call"):
    if T <= 0 or sigma <= 0:
        return {"delta":0, "gamma":0, "vega":0, "theta":0}
    d1 = (math.log(S/K) + (r+0.5*sigma**2)*T)/(sigma*math.sqrt(T))
    d2 = d1 - sigma*math.sqrt(T)
    pdf_d1 = norm.pdf(d1)
    if option_type=="call":
        delta = norm.cdf(d1)
        theta = - (S * pdf_d1 * sigma)/(2*math.sqrt(T)) - r*K*math.exp(-r*T)*norm.cdf(d2)
    else:
        delta = norm.cdf(d1)-1
        theta = - (S * pdf_d1 * sigma)/(2*math.sqrt(T)) + r*K*math.exp(-r*T)*norm.cdf(-d2)
    gamma = pdf_d1/(S*sigma*math.sqrt(T))
    vega = S * pdf_d1 * math.sqrt(T)
    return {"delta": delta, "gamma": gamma, "vega": vega, "theta": theta}

# -------- NSE Option Scan --------
def scan_nse_options(symbol, save_csv=None):
    print(f"\nFetching {symbol} option chain from NSE...")
    data = nse_optionchain_scrapper(symbol)
    underlying = float(data["records"]["underlyingValue"])
    expiry_dates = data["records"]["expiryDates"]
    nearest_expiry = expiry_dates[1]  # nearest expiry
    rows = []

    # Spot & historical vol fallback
    hist_spot, hist_sigma = historical_vol_from_yfinance(symbol)
    if hist_spot is None or hist_spot <= 0:
        hist_spot = underlying
    if hist_sigma is None or hist_sigma <= 0:
        hist_sigma = 0.20  # fallback

    # Prepare strikes
    data_rows = data["records"]["data"]
    strikes = sorted({float(x["strikePrice"]) for x in data_rows})
    atm_strike = min(strikes, key=lambda x: abs(x - hist_spot))
    atm_idx = strikes.index(atm_strike)
    low_idx = max(0, atm_idx - MAX_STRIKES_EACH_SIDE)
    high_idx = min(len(strikes)-1, atm_idx + MAX_STRIKES_EACH_SIDE)
    selected_strikes = strikes[low_idx:high_idx+1]

    # Auto-calibrate ATM vol
    atm_option = None
    for item in data_rows:
        if float(item["strikePrice"]) == atm_strike:
            for key in ["CE","PE"]:
                opt = item.get(key)
                if opt and opt.get("expiryDate")==nearest_expiry:
                    bid = float(opt.get("bidprice") or 0.0)
                    ask = float(opt.get("askPrice") or 0.0)
                    mid = (bid+ask)/2 if bid>0 and ask>0 else float(opt.get("lastPrice") or 0.0)
                    atm_option = {"mid": mid, "type": "call" if key=="CE" else "put"}
                    break
            if atm_option:
                break

    if atm_option:
        days = (dt.datetime.strptime(nearest_expiry,"%d-%b-%Y").date() - dt.date.today()).days
        T = max(days/365.0, 0.0001)
        sigma_atm = implied_volatility(atm_option["mid"], hist_spot, atm_strike, RISK_FREE_RATE, T, atm_option["type"])
        if math.isnan(sigma_atm) or sigma_atm<=0:
            sigma_atm = hist_sigma
    else:
        sigma_atm = hist_sigma

    # Scan strikes
    for item in data_rows:
        sp = float(item["strikePrice"])
        if sp not in selected_strikes:
            continue
        for key, opt_type in [("CE","call"),("PE","put")]:
            opt = item.get(key)
            if not opt or opt.get("expiryDate")!=nearest_expiry:
                continue
            last_price = float(opt.get("lastPrice") or 0.0)
            bid = float(opt.get("bidprice") or 0.0)
            ask = float(opt.get("askPrice") or 0.0)
            mid = (bid+ask)/2 if (bid>0 and ask>0) else last_price
            oi = int(opt.get("openInterest") or 0)
            volu = int(opt.get("totalTradedVolume") or 0)
            days = (dt.datetime.strptime(nearest_expiry,"%d-%b-%Y").date() - dt.date.today()).days
            T = max(days/365.0, 0.0001)
            model = bs_price(hist_spot, sp, RISK_FREE_RATE, sigma_atm, T, opt_type)
            imp_vol = implied_volatility(mid, hist_spot, sp, RISK_FREE_RATE, T, opt_type)
            signal, diff = decide_trade(mid, model, DECISION_THRESHOLD)
            greeks = bs_greeks(hist_spot, sp, RISK_FREE_RATE, sigma_atm, T, opt_type)
            rows.append({
                "expiry": nearest_expiry,
                "strike": sp,
                "optionType": opt_type,
                "bid": bid,
                "ask": ask,
                "midPrice": mid,
                "modelPrice": model,
                "impliedVol": imp_vol,
                "calibratedVol": sigma_atm,
                "delta": greeks["delta"],
                "gamma": greeks["gamma"],
                "vega": greeks["vega"],
                "theta": greeks["theta"],
                "signal": signal,
                "diff_pct": diff,
                "oi": oi,
                "volume": volu
            })

    df = pd.DataFrame(rows)
    df = df.sort_values(["strike","optionType"]).reset_index(drop=True)
    if save_csv:
        df.to_csv(save_csv, index=False)
    return {
        "symbol": symbol,
        "spot": hist_spot,
        "atm_vol": sigma_atm,
        "nearest_expiry": nearest_expiry,
        "df": df
    }

# -------- Example Run --------
if __name__=="__main__":
    tickers = ["NIFTY", "M&M"]  # Flexible: add any NSE ticker here
    for t in tickers:
        result = scan_nse_options(t, save_csv=f"{t}_options_scan.csv")
        df = result["df"]
        print(f"\n===== {t} OPTION SCAN =====")
        print(f"Spot (used)     : {result['spot']:.2f}")
        print(f"ATM calibrated vol : {result['atm_vol']:.4f}")
        print(f"Nearest expiry  : {result['nearest_expiry']}")
        print(f"Rows scanned    : {len(df)}")

        buys = df[df["signal"]=="BUY"].sort_values("diff_pct").head(10)
        sells = df[df["signal"]=="SELL"].sort_values("diff_pct",ascending=False).head(10)

        print("\nTop BUY opportunities (market < model):")
        print(buys[["strike","optionType","midPrice","modelPrice","diff_pct","oi","impliedVol","volume","delta","gamma","vega","theta"]].to_string(index=False))

        print("\nTop SELL opportunities (market > model):")
        print(sells[["strike","optionType","midPrice","modelPrice","diff_pct","oi","impliedVol","volume","delta","gamma","vega","theta"]].to_string(index=False))

        print(f"\nSaved results to: {t}_options_scan.csv")



Fetching NIFTY option chain from NSE...


ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: NIFTY"}}}
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['NIFTY']: YFTzMissingError('possibly delisted; no timezone found')



===== NIFTY OPTION SCAN =====
Spot (used)     : 25850.80
ATM calibrated vol : 0.1124
Nearest expiry  : 28-Oct-2025
Rows scanned    : 26

Top BUY opportunities (market < model):
 strike optionType  midPrice  modelPrice  diff_pct    oi  impliedVol  volume     delta    gamma        vega        theta
25650.0        put    49.325   78.622588 -0.372636 14441    0.089159  108011 -0.289180 0.000794 1308.300050 -2902.535203
25700.0        put    59.400   94.376363 -0.370605 79235    0.086509  352060 -0.330406 0.000842 1386.671103 -3038.667856
25600.0        put    41.200   64.884377 -0.365024 57673    0.092016  286479 -0.250474 0.000739 1217.392277 -2730.216680
25750.0        put    72.450  112.257886 -0.354611 12881    0.084530  134973 -0.373715 0.000880 1449.652077 -3131.966689
25550.0        put    34.425   53.029295 -0.350831 13033    0.094822   88657 -0.214634 0.000678 1117.140637 -2529.372908
25800.0        put    87.550  132.354977 -0.338521 57768    0.082252  541153 -0.418593 0.000908 

ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: M&M"}}}
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['M&M']: YFTzMissingError('possibly delisted; no timezone found')



===== M&M OPTION SCAN =====
Spot (used)     : 3596.70
ATM calibrated vol : 0.2701
Nearest expiry  : 25-Nov-2025
Rows scanned    : 26

Top BUY opportunities (market < model):
 strike optionType  midPrice  modelPrice  diff_pct  oi  impliedVol  volume     delta    gamma       vega       theta
 3350.0        put     0.000   29.064342 -1.000000   0         NaN       0 -0.171083 0.000833 287.003822 -354.297752
 3650.0        put   106.550  138.746894 -0.232055  34    0.198380      39 -0.524420 0.001305 449.784500 -494.343195
 3400.0        put    34.525   39.935598 -0.135483 524    0.253587     535 -0.219101 0.000968 333.669845 -407.177555
 3750.0        put   173.350  200.356279 -0.134791  15    0.203801      18 -0.647991 0.001217 419.255969 -422.180451
 3450.0        put    46.900   53.511085 -0.123546  92    0.252294     100 -0.273214 0.001090 375.691085 -452.220238
 3500.0        put    62.150   70.042716 -0.112684 609    0.250733     541 -0.332331 0.001190 410.218344 -485.745062
 3550.

# NSE BSM Scanner Output Interpretation

## Overview of Output
The script scanned **NIFTY** and **M&M** options for the nearest expiry and
compared market prices to BSM model prices.

### NIFTY Highlights
- **Spot**: 25,709.85  
- **ATM calibrated vol**: 0.1779  
- **Nearest expiry**: 28-Oct-2025  
- **Rows scanned**: 26  

#### Significant Observations
- **Puts** are massively undervalued relative to BSM model:
  - Example: Strike 25,400 put → Market: 20.78 vs Model: 130.93 → Diff: -84%
  - Strikes 25,400–25,850 show deviations of -68% to -84%
- **Calls** slightly overvalued relative to BSM model:
  - Example: Strike 25,400 call → Market: 553.55 vs Model: 474.16 → Diff: +16.7%
  - Deviations decrease for higher strikes
- Implied volatilities for puts are extremely low (0.05–0.08), reflecting illiquid pricing

#### BUY/SELL Signals
- **Top BUYs**: Deep OTM puts (market << model)
- **Top SELLs**: OTM calls (market > model)
- Market price deviations suggest **liquidity-driven pricing** rather than true arbitrage

---

### M&M Highlights
- **Spot**: 3,596.70  
- **ATM calibrated vol**: 0.2701  
- **Nearest expiry**: 25-Nov-2025  
- **Rows scanned**: 26  

#### Significant Observations
- OTM puts slightly undervalued (~10–23% deviation)
- Some calls slightly overvalued (~3–4%)
- Illiquid strikes show zero volume or zero mid price, indicating low trading activity

#### BUY/SELL Signals
- **Top BUYs**: OTM puts (market < model)
- **Top SELLs**: OTM calls (market > model)
- Deviations are smaller than NIFTY but still notable for certain strikes

---

## Key Takeaways
1. **Extreme deviations** in NIFTY puts indicate **illiquidity or stale quotes**, not actionable arbitrage.
2. **Calls** on NIFTY show moderate overpricing, which could be explored if liquidity allows.
3. M&M deviations are smaller but consistent with market pricing.
4. Using **put-call parity** or BSM normalization can help sanity-check extreme mispricings.

---

## Next Steps
1. **Filter illiquid options** more aggressively to avoid misleading signals.
2. **Normalize deviations** using ATM BSM volatility and put-call parity.
3. **Visualize deviations** for quick identification of mispriced strikes.
4. Consider **other expiry dates** or **shorter-dated options** for more actionable signals.
5. Potentially **integrate live streaming data** for intraday scans.
6. Use outputs as a **research and educational tool**, not as direct trading advice.

---

